## **A Quantum Enhanced Markov Chain Monte Carlo  Algorithm : Experiments & Simulation** 

Our aim in this project is to explore the idea of an **Quantum Enhanced** MCMC algorithm, which was proposed initially by *David Layden et al* in the this [article](https://arxiv.org/abs/2203.12497). Markov Chain Monte-Carlo is one of the fundamental tools available for a sampling tasks that arises in wide variety of computational problems. One specific area of interest is sampling from thermal distribution of Spin-Glass type hamiltonians at low tempeatures, where the energy landscape turns into a space of sparse energy minimas separated by wide barren plateaus making it difficult for usual algorithms to sample effectively.

$ \mathbf{E}(s) = \sum_{ij} s_i J_{ij} s_j + \sum_{j} h_j s_j $


###### We will refer to this paper https://arxiv.org/abs/2203.12497 as the 'original paper'  throughout rest of the text.

The usual approach to this task is through the **Metropolis Algorithm**, which essentially consisits of two steps :
1. **Proposal Step :**  We propose a transition from the current state *s* to another state *s'*, with some probability $\mathbf{Q}(s \rightarrow s')$
2. **Acceptance Step :** Based on the sampling inverse-temperature $\beta$, we accept or reject the proposed transition with the probability $\mathbf{T}(s \rightarrow s') = min(1, \frac{\mathbf{Q}(s \rightarrow s')}{\mathbf{Q}(s' \rightarrow s)}e^{-\beta (\mathbf{E}(s') - \mathbf{E}(s))})$

The above algorithm could be justified from the idea of detailed balance which says,
$$     \mathbf{\mu}(s) \mathbf{Q}(s \rightarrow s') \mathbf{T}(s \rightarrow s') = \mathbf{\mu}(s') \mathbf{Q}(s' \rightarrow s) \mathbf{T}(s' \rightarrow s) 
$$ 
However under ideal circumstances we asssume the proposal algorithm to be symmeteric on its transitions i.e $\mathbf{Q}(s \rightarrow s') = \mathbf{Q}(s' \rightarrow s)$

The expectation is that with sufficient iteration of the above algorithm i.e an MCMC chain of the accepted states ($ s_0 \rightarrow s_1 \rightarrow s_2 \rightarrow s_3$ . .. ) we can closely approximate the desired probability distribution $\mathbf{P}(s) = \frac{e^{-\beta \mathbf{E}(s)}}{\mathcal{Z}_{\beta}}$. However one significant caveat in this approach is that under a general setting we lack a good *'proposal mechanism'* that can effciently propose transitions which would be accepted with high enough probability and this considerably slows down the convergence of the MCMC chain to the desired distribution. Basic classical approcahes like the *uniform-sampling* where we pick randomly pick a state at uniform or *local-sampling* where we propose local changes often fail to sample the space effectively because either the states proposed have high energy difference or are too localised to the intial state. 

The aim with the *Quantum Enhanced MCMC* is to somehow enhance the proposal step such that it provides us with transitions that are more likely to be accepted, i.e the energy differnece between the initial and the proposed state is minimal. In the figure below, the essential difference in the approach to the proposition step of this algorithm has been highlighted.                                      

![proposition-strategy](DATA/figures/prop-strategy.jpeg)

###### Figures taken from https://arxiv.org/abs/2203.12497


Note how the proposed states are from is from a different energy minima and yet the energy difference between *s* and *s'* is low, compared to the *local* and *uniform* proposals it has both higher probability of being accepted and it enact sampling in a more global sense that is sees the entire energy landscape. Physically it could be thought analogous to *'quantum-mechanical-tunneliing'*, where particles can tunnel through narrow regions of high-energy barriers. In the next section we highlight the basic step of this enhanced proposition algorithm (refer as quMCMC)                                       


However, there exists sophisticated algoriths for special cases of the ising models that would be sampled dependeing upon their properties like connectivity, for example see Cluster-flipping algorithm, Swesden-Wang algorithm etc. but we will be excluding them from your discussions


#### **quMCMC**

The approach to built a quantum algorithm that can help us with better propositions can be designed based on your intuition of the available QAOA type algorithms where we stack layers of alternating operators applying the *Phase* and the *Mixer* Unitaries. There the role of the phase unitaries $\hat{U_p}$ is to apply phases corresponding the problem hamiltonian $\hat{H_p}$ and mixer unitary $\hat{U_m}$ acts to mix-up the phases of different basis states such as to allow transitions between them, the usual choice for being $ \hat{H}_m = \sum_i \hat{X}_i$ since it easier to implemnt on device and also does not commute with the $\hat{H}_p$. though for this algorithm we won't need to optimse the angles corresponding to this unitaries like the QAOA we use the same essential ingredeints.

Based on this ideas we construct an evolution hamiltonian of the form 
$$                      \hat{H}_{ev} = \gamma \hat{H}_p + \alpha (1- \gamma)  \hat{H}_m
$$

Here $\gamma$ introduced a relative weight between the two hamiltonians and $\alpha$ is introduced to balance the overall strength between the two hamiltonians . The basic idea to enhancing the proposal mechanism here is to, start with a random state *s* and evolve it under the $\hat{U}_{ev} = e^{-i \tau \hat{H}_{ev}}$ for a short period of time and then measure it in a computational basis to get the proposed state *s'*. Then *s'* is either accepted or rejected into the mcmc chain based on $\mathbf{T}(s|s')$.

In terms of the proposal probability we have $\mathbf{Q}(s \rightarrow s') = |\bra{s'}\hat{U}_{ev}\ket{s}|$, and since we know from the rules of unitary evolution that $|\bra{s}\hat{U}_{ev}\ket{s'}| = |\bra{s'}\hat{U}_{ev}\ket{s}|$, our assumption $\mathbf{Q}(s \rightarrow s') = \mathbf{Q}(s' \rightarrow s)$ remains true.

It should be noted that only the *proposition step* in our algorithm reqruies a quantum resources, while the *acceptance step* can be run classically. This is of the same flavour as conventional *Hybrid Algorithms* but with the strict difference that id doesn't involve any parameter tuning or gradient calculation. However, it is evident that to effectively use this algorithm one would need quantum devices that could be operated *dynamically* i.e to be able to measure and reinitiate them online. And since most available devices lack this feature we cannot directly implement this algorith as is on a real quantum device, this issue will be adressed in a later section. 


#### Experiments on Statevector Simulators

In this section we will present the resukts we obtained from our experiments on the implementation of the quMCMC algorithm on statevector simulators. The implementation was done in `qiskit` as well as in `qulacs` backends both of which provides efficient statevector simulators that could further enhanced from CUDA based GPU-Accelarators. However, for our purpose it seemed fit to use the `qulacs` backend as it provided a considerable speed-up for the particular range of qubits we experimented with. 

###### Refer to: [Qulacs-Doc](http://docs.qulacs.org/en/latest/intro/0_about.html), [Qiskit-Doc](https://qiskit.org/documentation/tutorials/simulators/1_aer_provider.html)

Our implementation follows the exact steps mentioned in the algorithm. Paramters like relative weigtht $\gamma$, evolution time $\tau$ and the level of trotterization are picked randomly on every single circuit executions, however there ranges  

Though statevector simulations are inherently costly operations, its a fine tool to have an idea of the expected behaviour of our algorithms since it isn't affected by the hardware peoperties like limited connectivity, readout errros and so on. Thus to examine the performance of our algorithm at greater depth, here we will try sampling ground state of a fully connected ising model (all spins interact with all other).

Here we will consider the the case of n= 7 and n= 15 spin systems. For each case we will randomly generate five different instances of the ising model by picking a interaction matrix $J$ and bias $h$ of random gaussain numbers, and their corresponding boltzmann distribution is generated by exhaustive sampling at a certain low temperature ( apprx $\frac{1}{T} \approx 1.025$)

We then run the *Quantum-Enhanced MCMC* and the *Classical-Uniform MCMC* on all the moels for sufficient number of iterations.

The first plot shows the mean convergence trend in KL Divergence of the distribution approximated from the MCMC chain against the number of iterations, the fill denotes standard-deviation over all different instances. It should be noted that calculating KL Divergence is a computationally intractable task for larger cases since it requires exhaustive computation over expponentially many configurations of the system and should be avoided in general (in the original paper they choose to inspect the *Spectral-Gap* instead), but we think it still gives more accurate information about the rate of convergence of the MC process which is an important for many applications.

The second plot the shows the overall statistics of the Acceptance Probability ($\mathbf{T}(s \rightarrow s')$). For this, we iterate over all transitions that were proposed in the MCMC chain and calculate the corresponding $\mathbf{T}(s \rightarrow s')$ and dump them to a list. The plot shows the statistics of all the values, intuitively we expect a better performing algorithm to have most transitions accepted and thus the ditribution concentraed around zero.


##### **N = 7**

![n7kldiv](DATA/figures/n7.kldiv.png) 


![n7acpp](DATA/figures/n7.acpp.png) 

##### **N= 15**

![n15kldiv](DATA/figures/n15.kldiv.png)

![n15acpp](DATA/figures/n15.acpp.png)


There are some very sharp contrasts between the n= 7 and n= 15 case. Most specifically the contrast in the performance between the Classical and Quantum MCMC becomes significant with the increase in the number of spins. 

For the n= 7 case, both the MCMCc chains converge equally fast. The distribution of acceptance probability however shows some disticntion, and it is evident that the quantum-enhanced approach manages to propose states which are more likely to be accepted by the algorithm. For the n= 15 case the difference in the rate of convergence is drastic even with a minimal number of mcmc iterations.Similarly for the statistics of acceptance probability their is a wide gap in the peaks of their distribution, note also the low number of proposed transitions with acceptability close to one.

Though the running these simulation for further iterations and higher number of spins was in your best interest, we lacked the necessary resources for such ventures. But we think this provides us with raw evidence that the 'quantum-advantage' would only yield itself at higher number of spins.

#### Experiment on Real Quantum Hardware

Next we illustrate the results we obtained from running our experiments on a Real Quantum Hardware. F
As indicated above, the margin of quantum advantage becomes prominent only when we move to higher number of qubits. 

![ibmq-guadalupe-errmap](DATA/figures/ibmq_gdlp_errormap.png)

We were given access to the device ibmq-guadalupe (16 qubits) with max. 300 circuits to run and 100000 shots per job. Although, we intended to explore a 2D-Ising model with large number of qubits, based on the given specifications we had to restrict our model accordingly. 

As discussed previously current day quantum computers lack dynamic control making infeasible to implement the algorithm on a near-term device. Instead
we will use the 'offline' method described in the section III of the original paper, where we estimate the proposal matrix first by running large number of circuits with various values of $\gamma$,t ($\gamma \in (0.26,0.6), t \in (1.6,9.6)$) and SPAM twirlings, and averaging over the whole data. And then reconstructing the MCMC chain at a deisred temperature classically. The whole point is to separate the part we need to run on the quantum computer from the classical processing.

 Since we could only measure $10^5$ shots, number of spins had to be limited to 8 so that we could sample each transition atleast once (scales as $2^n \times 2^n$). 

Next we choose a model that respects the device connectivity, so that we dont require SWAP gates. The qubits 0,1,2,3,4,6,7,10 were chosen based on their lower CNOT & measurement error rates. The qubit to device mapping is shown below

![ibmq-guadalupe-qmap](DATA/figures/ibmq_gdlp_qubitmap.png)

The Ising model was chosen as follows(shown as a graph). The coupling and local magnetic fields were chosen randomly, while respecting connectivity.

![graph-ising](DATA/figures/model-interaction-map.png)

In the above figure the edege annotations depict corresponding interaction strength and the color indicates the corresponfing biases

__model more info

Using the device data we get the following proposal matrix (log scale). We attempted SPAM twirling as mentioned in  https://arxiv.org/abs/2203.12497, but were unable to execute the two-qubit twirling properly. This probably leads to the assymmetry in the *proposal matrix* $\mathbf{Q}(s|s')$ that we see here.

![pmatrix_fromdevice](DATA/figures/proposal_matrix_fromdevice.png)

For a given matrix $M = \{m_{jk}\}$ we use the following measure to quantify the assymmetry as suggested in the orignal paper,
$$
\chi^2 = \sum_{j<k} \frac{(m_{jk} - m_{kj})^2}{m_{jk} + m_{kj}} = 7255.5
$$


From our experience with the data collection process we have reasons to be concerned about the symmetry in the *proposal matrix*, and certainly our results are not very much in accord with the expected results mentioned in the orignal paper. 

In their work the authors generated the *transition probability matrix* $\mathbf{T}(s|s')$ from the collected data and further claims of convergence were based upon investigating the *spectral-gap* ($\delta$) of the same.

But given the circumstances and lack of resources to regenerate the requried data we will proceed in a slightly different fashion. Since the ising model in question is a 8-spin system we can easily construct the corresponding boltzmann ditribution exhaustively, and thus compute metircs like KL Divergence which are based on the exact distribution.

Next we constructed the MCMC chain from the obtained propotion matrix $\mathbf{Q}$ by using as a lookup table for the *proposal step* as indicated in the original paper. Here we will construct the MCMC chain for three different temperatures to get a better undertanding possible enhancements the algorithm provides. 

Also for the sake of comparison we will be comparing its performance with a classical-uniform MCMC and a simulated version of the quantum-enhanced MCMC

For our convenience we refer to something as 'offline' if the MCMC chain is built out of pre-sampled transition matrices as described above(** i.e the data is obtained from a real quantum hardware), and 'online' to refer to usual MCMC chains alternating between 'quantum' and 'classical' subroutines (**For our case 'online' results come from a simulator).

##### **Low Temperature | beta = 1.85**

![esm](DATA/figures/esmb1.png)

![acp1](DATA/figures/accp1.png)

![kldv1](DATA/figures/kldiv1.png)

![mag1](DATA/figures/mag1.png)

##### **Low Temperature | beta = 2.72**

![esmb2](DATA/figures/esmb2.png)

![accp2](DATA/figures/accp2.png)

![kldv2](DATA/figures/kldiv2.png)

![mag2](DATA/figures/mag2.png)

##### **High Temperature | beta = 0.55**

![esmb3](DATA/figures/esmb3.png)

![acp3](DATA/figures/accp3.png)

![kldv3](DATA/figures/kldiv3.png)

![mag3](DATA/figures/mag3.png)

Some takeaways from the above data,

As is expected from usual MCMC sampling methods its efficiency increases with temperature. This can be seen by comparing the convergence of KL Divergence for the high temperature regime to the other results.

A major issue to note here is that even for the low temperature cases, there is no significant improvement in performace of the the *quantum (offline)* sampling ethod with respect to *classical-uniform* sampling. Whereas the *quantum (online)* sampling given much better perfromance compared both the other methods. This disticntion between the performacne of *quantum (offline)* and *quantum (online)* approaches is quiet unsettling, since ideally we expected them to behave analogously.

Though we don't have a solid undertanding to what could have led to this anomaly, our best guess is that this stems from the errors in the associated in sampling entries of the proposal matrix $\mathbf{Q}(s|s')$. Since our experiments were conducted on very limited resources we have certainly less data than required to construct $\mathbf{Q}(s|s')$. 

From a computational perspective the whole *offline* scheme is highly inefficient, since we would require exponentially many samples to estimate it to slightest accuracy. Moreover the *online* method enjoys the benefits from the fact that after a few iterations the MCMC chain localizes itself on states with high probability which ensures a faster convergence to the desired thermal distribution with minimal computational effort.


 